# Cycle Time 예측

## 라이브러리 import

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from glob import glob
import os, random, time, gc, warnings

from tqdm import tqdm_notebook

import lightgbm as lgbm
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import mean_squared_error

from sklearn.feature_selection import RFECV


from sklearn.cluster import KMeans

from datetime import datetime

from math import sqrt
import io

warnings.filterwarnings('ignore')

pd.set_option('max_columns', 500)
pd.set_option('max_rows', 500)

%matplotlib inline

## 데이터 전처리

### 데이터 확인

In [2]:
#데이터 불러오기
df=pd.read_csv('D:\\test2\\data-0410-table7.csv',
                #names=['id','Uid','completed','CycleNumber','MCIn1',
                #       'MC1PrTime','MCOut1','MCIn2','MC2PrTime','MCOut2','MCIn3','MC3PrTime','MCOut3']
               )

print(df.shape)
df.head()

(290, 13)


,id,Uid,completed,CycleNumber,MCIn1,MC1Prtime,MCOut1,MCIn2,MC2Prtime,MCOut2,MCIn3,MC3Prtime,MCOut3
0,1,c37ba1af,1,1,0.00000,0.13,67754.00002,67757.00000,4.18,67786.99998,67789.99996,0.08,67792.00003
1,2,c37ba1af,1,1,67986.00000,0.76,67989.99997,67839.99998,4.56,68095.99999,68104.00002,1.07,68108.00000
2,3,c37ba1af,1,1,68123.99998,3.53,68135.99999,0.00000,0.32,68247.99996,68253.00002,0.16,68263.00004
3,4,c37ba1af,1,1,68784.99998,6.03,68949.99996,68958.99999,0.01,69066.99999,69082.99998,0.17,69100.00004
4,5,c37ba1af,1,1,0.00000,1.03,69120.00000,69122.99998,5.14,69269.99999,69277.00003,3.26,69282.00000


In [3]:
df.describe()

,id,completed,CycleNumber,MCIn1,MC1Prtime,MCOut1,MCIn2,MC2Prtime,MCOut2,MCIn3,MC3Prtime,MCOut3
count,290.000000,290.000000,290.000000,290.000000,290.000000,290.000000,290.000000,290.000000,290.000000,290.000000,290.000000,290.000000
mean,313.541379,0.968966,3.031034,42824.944826,3.693759,65596.468968,54680.806897,4.170207,68133.941382,70040.531035,4.255345,69281.517243
std,184.689902,0.173711,1.484313,35070.705099,3.436148,19860.252184,30381.457399,4.034612,15535.097562,10445.941416,4.062186,12619.032607
min,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000
25%,105.250000,1.000000,1.000000,0.000000,1.182500,68864.250018,67757.499995,1.277500,69132.250030,69209.999963,1.072500,69157.499978
50%,351.500000,1.000000,3.000000,68687.499960,2.810000,71481.500010,70079.500040,2.925000,71681.499980,71745.000005,3.180000,71669.499970
75%,443.750000,1.000000,4.000000,72147.749997,5.177500,73385.750005,72844.499967,5.600000,73509.500007,73424.749967,6.370000,73389.250022
max,613.000000,1.000000,5.000000,75074.999990,23.040000,75095.000040,75065.999960,24.580000,75159.999960,75202.000040,27.230000,75146.999960


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           290 non-null    int64  
 1   Uid          290 non-null    object 
 2   completed    290 non-null    int64  
 3   CycleNumber  290 non-null    int64  
 4   MCIn1        290 non-null    float64
 5   MC1Prtime    290 non-null    float64
 6   MCOut1       290 non-null    float64
 7   MCIn2        290 non-null    float64
 8   MC2Prtime    290 non-null    float64
 9   MCOut2       290 non-null    float64
 10  MCIn3        290 non-null    float64
 11  MC3Prtime    290 non-null    float64
 12  MCOut3       290 non-null    float64
dtypes: float64(9), int64(3), object(1)
memory usage: 29.6+ KB


In [5]:
#df_dropped_3.info()
df_dropped_3 = df

In [6]:
#열별로 결측값 개수 확인
count1=0

for i in df_dropped_3:
    count2 = 0
    for j in df_dropped_3[i]:
        if j == 0:
            count2+=1
    print('-'*50)
    print(i,"열의 결측값 개수:",count2)
    count1+=count2
    
print('-'*50)
print('전체 결측치:',count1)

--------------------------------------------------
id 열의 결측값 개수: 0
--------------------------------------------------
Uid 열의 결측값 개수: 0
--------------------------------------------------
completed 열의 결측값 개수: 9
--------------------------------------------------
CycleNumber 열의 결측값 개수: 0
--------------------------------------------------
MCIn1 열의 결측값 개수: 116
--------------------------------------------------
MC1Prtime 열의 결측값 개수: 2
--------------------------------------------------
MCOut1 열의 결측값 개수: 24
--------------------------------------------------
MCIn2 열의 결측값 개수: 68
--------------------------------------------------
MC2Prtime 열의 결측값 개수: 0
--------------------------------------------------
MCOut2 열의 결측값 개수: 14
--------------------------------------------------
MCIn3 열의 결측값 개수: 6
--------------------------------------------------
MC3Prtime 열의 결측값 개수: 3
--------------------------------------------------
MCOut3 열의 결측값 개수: 9
--------------------------------------------------
전체 결측치: 251


### 이상치 제거

#### 함수 선언

In [161]:
#열의 값 중 시간 순서가 맞지 않는 값이 있는지 확인하는 함수 선언
def check_col_order(df,col): #데이터 프레임이랑 열의 이름을 인자로 받음
    count=0
    print('-'*70)
    print('열 내 순서 확인 시작')
    for i,j in enumerate(df[col]):
        if i == len(df)-1:
            print('마지막 행(',i,')은 비교불가')
            break
        if (df[col][i] != 0) & (df[col][i+1] != 0) :
            if df[col][i] > df[col][i+1]:
                print('인덱스:',i)
                count+=1
    print('이상값:',count)
    print('열 내 순서 확인 완료')
    print('-'*70)
    

    
#열의 값 중 시간 순서가 맞지 않는 값을 제거하는 함수 선언
def check_col_order_del(df,col): #데이터 프레임이랑 열의 이름을 인자로 받음
    print('-'*70)
    print('제거 시작')
    for i,j in enumerate(df[col]):
        if i == len(df)-1:
            print('마지막 행(',i,')은 비교불가')
            break
        if df[col][i] != 0:
            if df[col][i] > df[col][i+1]:
                print('인덱스-',i,'제거')
                df[col][i] = 0
    print('제거 완료')
    print('-'*70)
    
    
    
#열의 값 중 시간 순서가 맞지 않는 값의 다음 행을 제거하는 함수 선언
def check_col_order_del2(df,col): #데이터 프레임이랑 열의 이름을 인자로 받음
    print('-'*70)
    print('제거 시작')
    for i,j in enumerate(df[col]):
        if i == len(df)-1:
            print('마지막 행(',i,')은 비교불가')
            break
        if df[col][i] != 0:
            if df[col][i] > df[col][i+1]:
                print('인덱스-',i+1,'제거')
                df[col][i+1] = 0
    print('제거 완료')
    print('-'*70)
    
    
    
#열의 값 중 시간 순서가 맞지 않는 값의 다음 행과 그 다음 행을 제거하는 함수 선언
def check_col_order_del3(df,col): #데이터 프레임이랑 열의 이름을 인자로 받음
    print('-'*70)
    print('제거 시작')
    for i,j in enumerate(df[col]):
        if i == len(df)-1:
            print('마지막 행(',i,')은 비교불가')
            break
        if df[col][i] != 0:
            if df[col][i] > df[col][i+1]:
                print('인덱스-',i+1,i+2,'제거')
                df[col][i+1] = 0
                df[col][i+2] = 0
    print('제거 완료')
    print('-'*70)
    
    
    
#열의 값 중 시간 순서가 맞지 않는 값의 다음 행과 그 다음 행과 그 다음행을 제거하는 함수 선언
def check_col_order_del4(df,col): #데이터 프레임이랑 열의 이름을 인자로 받음
    print('-'*70)
    print('제거 시작')
    for i,j in enumerate(df[col]):
        if i == len(df)-1:
            print('마지막 행(',i,')은 비교불가')
            break
        if df[col][i] != 0:
            if df[col][i] > df[col][i+1]:
                print('인덱스-',i+1,i+2,i+3,'제거')
                df[col][i+1] = 0
                df[col][i+2] = 0
                df[col][i+3] = 0
    print('제거 완료')
    print('-'*70)
    


#열의 값 중 시간 순서가 맞지 않는 값의 다음 행과 그 전 행을 제거하는 함수 선언
def check_col_order_del_updown(df,col): #데이터 프레임이랑 열의 이름을 인자로 받음
    print('-'*70)
    print('제거 시작')
    for i,j in enumerate(df[col]):
        if i == len(df)-1:
            print('마지막 행(',i,')은 비교불가')
            break
        if df[col][i] != 0:
            if df[col][i] > df[col][i+1]:
                print('인덱스-',i-1,i,i+1,'제거')
                df[col][i+1] = 0
                df[col][i-1] = 0
                df[col][i] = 0
    print('제거 완료')
    print('-'*70)
    
    
        
#MCOut + MCPrtime 이 다음 행 MCOut의 값 보다 작은 행 확인하는 함수 선언
def check_outPrtime(df, prtime, out): #데이터 프레임, prtime, out을 인자로 받음
    try :
        print('-'*70)
        print('MCOut + MCPrtime 확인 시작')
        count=0
        for i, j in enumerate(df[out]):
            if (df[prtime][i] + j) > df[out][i+1]:
                print('-'*70)
                print(i,'번째 행 확인바람')
                print(str(prtime),'+',str(out),':',df[prtime][i] + j,'다음행',str(out),':',df[out][i+1])
                count+=1
    except :
        print('-'*70)
        print(i+1,'=',len(df),'이면 정상실행')
        print('이상값:',count)
        print('MCOut + MCPrtime 확인 완료')
        print('-'*70)
        pass
    
    
    
#MCOut + MCPrtime 이 다음 행 MCOut의 값 보다 작은 행에 값을 더해주는 함수 선언
def check_outPrtime_sum(df, prtime, out): #데이터 프레임, prtime, out을 인자로 받음
    print('-'*70)
    try :
        print('-'*70)
        print('sum 시작')
        for i, j in enumerate(df[out]):
            if (df[prtime][i] + j) > df[out][i+1]:
                #일단 1을 더했습니다
                df[out][i+1] = df[prtime][i] + j + 1
    except :
        print(i+1,'=',len(df),'이면 정상실행')
        print('sum 완료')
        print('-'*70)
        pass
    
    
    
# MCOut2 + MC2PrTime < MCIn3 이 아닌 값 확인하는 함수 선언
def check_outPrtimeIn(df, out, prtime, inn):
    print('-'*70)
    print('확인 시작')
    count=0
    for i, j in enumerate(df[inn]):
        if (df[out][i] + df[prtime][i]) >= df[inn][i]:
            print('-'*70)
            print(i,'번째 행 확인바람')
            print('MCOut+MCPrTime:',df[out][i] + df[prtime][i]
                 ,'MCIn:',df[inn][i])
            count += 1    
    print('-'*70)
    print('이상값:',count)
    print('확인 완료')
    print('-'*70)
    
    
    
# MCOut2 + MC2PrTime < MCIn3 이 아닌 행의 MCOut2 값 0으로 대체하는 함수 선언
def check_outPrtimeIn_del(df, out, prtime, inn):
    print('-'*70)
    print('제거 시작')
    count=0
    for i, j in enumerate(df[inn]):
        if (df[out][i] != 0) & (df[inn][i] != 0) :
            if (df[out][i] + df[prtime][i]) >= df[inn][i]:
                df[out][i] = 0
    print('제거 완료')
    print('-'*70)
    
    

#MAX, MIN비교해서 ran값으로 입력 하는 함수 선언
def check_minmax_ran(df, col, list_left_2, list_right_2):
    count=0
    for i, j in enumerate(df[col]):
        if j == 0:
            #위 max 구하기
            list_up=[]
            for k in range(0, i):
                list_up.append(df[col][k])
            list_up_2 = filter(lambda a: a != 0, list_up)
            up=max(list_up_2)
            print('-'*50)
            print(i,"행 입니다.")
            print('up:', up)
        
            #왼쪽 max 구하기
            list_left=[]
            for k in list_left_2:
                list_left.append(df[k][i])
            list_left_3=list(filter(lambda a: a != 0, list_left))
            if list_left_3 == []:
                left = up
            else :
                left=max(list_left_3)
            print('left:',left)
        
            #오른쪽 min 구하기
            list_right=[]
            for k in list_right_2:
                list_right.append(df[k][i])
            list_right_3=filter(lambda a: a != 0, list_right)
            right=min(list_right_3)
            print('right:',right)
        
            #밑 min 구하기
            list_down=[]
            for k in range(i+1, len(df[col])-1):
                list_down.append(df[col][k])
            list_down_2=set(list_down) #중복값 삭제
            try :
                list_down_2.remove(0) #min이 0인 경우는 삭제해서 생각 하지않기
            except :
                pass
            try :
                down=min(list_down_2)
            except :
                print('마지막 행(',i,')입니다.')
            print('down:',down)
        
            # MAX, MIN비교해서 랜덤 값으로 대체
            try :
                df[col][i] = np.random.randint(max(up, left),min(right, down), size=1)
            except :
                count+=1
                print('해당 행은 무시(',count,')')
                df[col][i] = np.random.randint(min(up, left, right, down),max(up, left, right, down), size=1)
                pass

#### MCOut3 결측치 채우기

In [8]:
#열별로 결측값 개수 확인
count1=0

for i in df_dropped_3:
    count2 = 0
    for j in df_dropped_3[i]:
        if j == 0:
            count2+=1
    print('-'*50)
    print(i,"열의 결측값 개수:",count2)
    count1+=count2
    
print('-'*50)
print('전체 결측치:',count1)

--------------------------------------------------
id 열의 결측값 개수: 0
--------------------------------------------------
Uid 열의 결측값 개수: 0
--------------------------------------------------
completed 열의 결측값 개수: 9
--------------------------------------------------
CycleNumber 열의 결측값 개수: 0
--------------------------------------------------
MCIn1 열의 결측값 개수: 116
--------------------------------------------------
MC1Prtime 열의 결측값 개수: 2
--------------------------------------------------
MCOut1 열의 결측값 개수: 24
--------------------------------------------------
MCIn2 열의 결측값 개수: 68
--------------------------------------------------
MC2Prtime 열의 결측값 개수: 0
--------------------------------------------------
MCOut2 열의 결측값 개수: 14
--------------------------------------------------
MCIn3 열의 결측값 개수: 6
--------------------------------------------------
MC3Prtime 열의 결측값 개수: 3
--------------------------------------------------
MCOut3 열의 결측값 개수: 9
--------------------------------------------------
전체 결측치: 251


In [9]:
# MCOut3 열을 기준으로 오름차순 정렬
df_sort_mcout3=df_dropped_3.sort_values(by=['MCOut3'], axis=0, ascending=True)

# 인덱스 리셋
df_sort_mcout3 = df_sort_mcout3.reset_index(drop=True)
df_sort_mcout3

,id,Uid,completed,CycleNumber,MCIn1,MC1Prtime,MCOut1,MCIn2,MC2Prtime,MCOut2,MCIn3,MC3Prtime,MCOut3
0,613,c37b877b,0,5,74964.00001,1.42,74970.99996,71211.00001,0.57,75144.99997,0.00000,1.06,0.00000
1,463,c37b75d8,0,4,75074.99999,4.17,75095.00004,74981.99998,3.41,75159.99996,0.00000,1.95,0.00000
2,447,c37b81a9,0,4,75072.00001,7.23,75083.00003,0.00000,2.26,75155.00000,0.00000,3.03,0.00000
3,371,c37ba2e1,0,3,0.00000,1.03,75062.99998,75065.99996,3.15,75143.99998,0.00000,10.72,0.00000
4,343,c37b770b,0,3,0.00000,6.61,0.00000,0.00000,2.50,0.00000,75178.00002,0.57,0.00000
5,298,c37b7dfe,0,3,0.00000,1.76,75049.99998,75064.99997,1.09,75135.00004,0.00000,1.70,0.00000
6,267,c37b997a,0,3,74615.99999,1.33,75047.99999,74628.00000,7.44,74997.99996,75202.00004,6.39,0.00000
7,118,c37b7e02,0,1,0.00000,1.21,74657.99998,74659.99997,1.19,74691.00003,0.00000,2.43,0.00000
8,72,c37b9728,0,1,75032.00001,6.93,75043.00002,75047.00000,7.16,75126.00001,75193.00001,4.28,0.00000
9,401,c37b9987,1,4,67710.00004,2.59,67714.00001,67717.99999,0.22,67721.99996,67731.99998,1.71,67733.99997


In [10]:
#mcout3 순서확인
check_col_order(df_sort_mcout3, 'MCOut3')

----------------------------------------------------------------------
열 내 순서 확인 시작
마지막 행( 289 )은 비교불가
이상값: 0
열 내 순서 확인 완료
----------------------------------------------------------------------


In [11]:
df_sort_mcout3

,id,Uid,completed,CycleNumber,MCIn1,MC1Prtime,MCOut1,MCIn2,MC2Prtime,MCOut2,MCIn3,MC3Prtime,MCOut3
0,613,c37b877b,0,5,74964.00001,1.42,74970.99996,71211.00001,0.57,75144.99997,0.00000,1.06,0.00000
1,463,c37b75d8,0,4,75074.99999,4.17,75095.00004,74981.99998,3.41,75159.99996,0.00000,1.95,0.00000
2,447,c37b81a9,0,4,75072.00001,7.23,75083.00003,0.00000,2.26,75155.00000,0.00000,3.03,0.00000
3,371,c37ba2e1,0,3,0.00000,1.03,75062.99998,75065.99996,3.15,75143.99998,0.00000,10.72,0.00000
4,343,c37b770b,0,3,0.00000,6.61,0.00000,0.00000,2.50,0.00000,75178.00002,0.57,0.00000
5,298,c37b7dfe,0,3,0.00000,1.76,75049.99998,75064.99997,1.09,75135.00004,0.00000,1.70,0.00000
6,267,c37b997a,0,3,74615.99999,1.33,75047.99999,74628.00000,7.44,74997.99996,75202.00004,6.39,0.00000
7,118,c37b7e02,0,1,0.00000,1.21,74657.99998,74659.99997,1.19,74691.00003,0.00000,2.43,0.00000
8,72,c37b9728,0,1,75032.00001,6.93,75043.00002,75047.00000,7.16,75126.00001,75193.00001,4.28,0.00000
9,401,c37b9987,1,4,67710.00004,2.59,67714.00001,67717.99999,0.22,67721.99996,67731.99998,1.71,67733.99997


In [12]:
check_outPrtime(df_sort_mcout3, 'MC3Prtime', 'MCOut3')
check_outPrtime_sum(df_sort_mcout3, 'MC3Prtime', 'MCOut3')

----------------------------------------------------------------------
MCOut + MCPrtime 확인 시작
----------------------------------------------------------------------
0 번째 행 확인바람
MC3Prtime + MCOut3 : 1.06 다음행 MCOut3 : 0.0
----------------------------------------------------------------------
1 번째 행 확인바람
MC3Prtime + MCOut3 : 1.95 다음행 MCOut3 : 0.0
----------------------------------------------------------------------
2 번째 행 확인바람
MC3Prtime + MCOut3 : 3.03 다음행 MCOut3 : 0.0
----------------------------------------------------------------------
3 번째 행 확인바람
MC3Prtime + MCOut3 : 10.72 다음행 MCOut3 : 0.0
----------------------------------------------------------------------
4 번째 행 확인바람
MC3Prtime + MCOut3 : 0.57 다음행 MCOut3 : 0.0
----------------------------------------------------------------------
5 번째 행 확인바람
MC3Prtime + MCOut3 : 1.7 다음행 MCOut3 : 0.0
----------------------------------------------------------------------
6 번째 행 확인바람
MC3Prtime + MCOut3 : 6.39 다음행 MCOut3 : 0.0
------------------------

In [13]:
check_col_order(df_sort_mcout3, 'MCOut3')

----------------------------------------------------------------------
열 내 순서 확인 시작
마지막 행( 289 )은 비교불가
이상값: 0
열 내 순서 확인 완료
----------------------------------------------------------------------


In [14]:
df_sort_mcout3

,id,Uid,completed,CycleNumber,MCIn1,MC1Prtime,MCOut1,MCIn2,MC2Prtime,MCOut2,MCIn3,MC3Prtime,MCOut3
0,613,c37b877b,0,5,74964.00001,1.42,74970.99996,71211.00001,0.57,75144.99997,0.00000,1.06,0.00000
1,463,c37b75d8,0,4,75074.99999,4.17,75095.00004,74981.99998,3.41,75159.99996,0.00000,1.95,2.06000
2,447,c37b81a9,0,4,75072.00001,7.23,75083.00003,0.00000,2.26,75155.00000,0.00000,3.03,5.01000
3,371,c37ba2e1,0,3,0.00000,1.03,75062.99998,75065.99996,3.15,75143.99998,0.00000,10.72,9.04000
4,343,c37b770b,0,3,0.00000,6.61,0.00000,0.00000,2.50,0.00000,75178.00002,0.57,20.76000
5,298,c37b7dfe,0,3,0.00000,1.76,75049.99998,75064.99997,1.09,75135.00004,0.00000,1.70,22.33000
6,267,c37b997a,0,3,74615.99999,1.33,75047.99999,74628.00000,7.44,74997.99996,75202.00004,6.39,25.03000
7,118,c37b7e02,0,1,0.00000,1.21,74657.99998,74659.99997,1.19,74691.00003,0.00000,2.43,32.42000
8,72,c37b9728,0,1,75032.00001,6.93,75043.00002,75047.00000,7.16,75126.00001,75193.00001,4.28,35.85000
9,401,c37b9987,1,4,67710.00004,2.59,67714.00001,67717.99999,0.22,67721.99996,67731.99998,1.71,67733.99997


In [15]:
#불완전 데이터 삭제
df_sort_mcout3 = df_sort_mcout3.drop([0,1,2,3,4,5,6,7,8])
df_sort_mcout3 = df_sort_mcout3.reset_index(drop=True)
df_sort_mcout3

check_col_order(df_sort_mcout3, 'MCOut3')
check_outPrtime(df_sort_mcout3, 'MC3Prtime', 'MCOut3')

----------------------------------------------------------------------
열 내 순서 확인 시작
마지막 행( 280 )은 비교불가
이상값: 0
열 내 순서 확인 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
MCOut + MCPrtime 확인 시작
----------------------------------------------------------------------
281 = 281 이면 정상실행
이상값: 0
MCOut + MCPrtime 확인 완료
----------------------------------------------------------------------


In [16]:
# MCIn3 열 내 순서 확인
check_col_order(df_sort_mcout3, 'MCIn3')

----------------------------------------------------------------------
열 내 순서 확인 시작
인덱스: 100
인덱스: 122
인덱스: 206
인덱스: 209
인덱스: 253
마지막 행( 280 )은 비교불가
이상값: 5
열 내 순서 확인 완료
----------------------------------------------------------------------


In [17]:
# MCIn3 열 내 순서 맞지 않는 값 제거
check_col_order_del(df_sort_mcout3, 'MCIn3')

----------------------------------------------------------------------
제거 시작
인덱스- 100 제거
인덱스- 122 제거
인덱스- 206 제거
인덱스- 209 제거
인덱스- 253 제거
마지막 행( 280 )은 비교불가
제거 완료
----------------------------------------------------------------------


In [18]:
check_col_order(df_sort_mcout3, 'MCIn3')

----------------------------------------------------------------------
열 내 순서 확인 시작
마지막 행( 280 )은 비교불가
이상값: 0
열 내 순서 확인 완료
----------------------------------------------------------------------


In [19]:
# MCOut2 + MC2Prtime < MCIn3 이 아닌 값 확인
check_outPrtimeIn(df_sort_mcout3, 'MCOut2', 'MC2Prtime', 'MCIn3')

----------------------------------------------------------------------
확인 시작
----------------------------------------------------------------------
3 번째 행 확인바람
MCOut+MCPrTime: 67791.17997999999 MCIn: 67789.99996
----------------------------------------------------------------------
8 번째 행 확인바람
MCOut+MCPrTime: 67919.38997 MCIn: 67907.99998
----------------------------------------------------------------------
14 번째 행 확인바람
MCOut+MCPrTime: 68059.36002000001 MCIn: 68044.99997
----------------------------------------------------------------------
24 번째 행 확인바람
MCOut+MCPrTime: 68153.71999 MCIn: 68151.99997
----------------------------------------------------------------------
25 번째 행 확인바람
MCOut+MCPrTime: 68256.67 MCIn: 68252.00003
----------------------------------------------------------------------
27 번째 행 확인바람
MCOut+MCPrTime: 68270.01004 MCIn: 68266.00002
----------------------------------------------------------------------
47 번째 행 확인바람
MCOut+MCPrTime: 68860.17997 MCIn: 68857.00004
------

In [20]:
df_sort_mcout3

,id,Uid,completed,CycleNumber,MCIn1,MC1Prtime,MCOut1,MCIn2,MC2Prtime,MCOut2,MCIn3,MC3Prtime,MCOut3
0,401,c37b9987,1,4,67710.00004,2.59,67714.00001,67717.99999,0.22,67721.99996,67731.99998,1.71,67733.99997
1,601,c37b877b,1,5,67721.99996,4.38,67728.00001,0.00000,2.52,67747.99997,67753.00002,8.22,67762.99996
2,426,c37b81a9,1,4,0.00000,4.91,67734.99996,67738.00003,1.18,67756.00000,67767.00002,2.30,67772.21996
3,1,c37ba1af,1,1,0.00000,0.13,67754.00002,67757.00000,4.18,67786.99998,67789.99996,0.08,67792.00003
4,51,c37b9728,1,1,67719.99997,10.40,0.00000,67724.00004,0.62,67737.00004,67809.00001,1.61,67811.99999
5,76,c37b7970,1,1,0.00000,0.50,0.00000,67758.99998,7.21,67804.00004,67812.99998,9.25,67825.99999
6,326,c37b770b,1,3,0.00000,5.38,67792.00003,0.00000,2.39,67880.99998,67887.00003,2.18,67892.00000
7,376,c37ba1b6,1,4,0.00000,0.49,67794.00002,0.00000,1.30,67883.99996,67893.99998,4.03,67903.00001
8,602,c37b877b,1,5,0.00000,4.60,67800.99997,67802.99996,10.39,67908.99997,67907.99998,3.16,67912.00004
9,427,c37b81a9,1,4,67804.00004,0.10,67806.00003,67813.99998,2.44,67916.00002,67927.00003,3.31,67931.00001


In [21]:
# MCOut2 + MC2Prtime < MCIn3 이 아닌 값 제거
check_outPrtimeIn_del(df_sort_mcout3, 'MCOut2', 'MC2Prtime', 'MCIn3')

----------------------------------------------------------------------
제거 시작
제거 완료
----------------------------------------------------------------------


In [22]:
df_sort_mcout3

,id,Uid,completed,CycleNumber,MCIn1,MC1Prtime,MCOut1,MCIn2,MC2Prtime,MCOut2,MCIn3,MC3Prtime,MCOut3
0,401,c37b9987,1,4,67710.00004,2.59,67714.00001,67717.99999,0.22,67721.99996,67731.99998,1.71,67733.99997
1,601,c37b877b,1,5,67721.99996,4.38,67728.00001,0.00000,2.52,67747.99997,67753.00002,8.22,67762.99996
2,426,c37b81a9,1,4,0.00000,4.91,67734.99996,67738.00003,1.18,67756.00000,67767.00002,2.30,67772.21996
3,1,c37ba1af,1,1,0.00000,0.13,67754.00002,67757.00000,4.18,0.00000,67789.99996,0.08,67792.00003
4,51,c37b9728,1,1,67719.99997,10.40,0.00000,67724.00004,0.62,67737.00004,67809.00001,1.61,67811.99999
5,76,c37b7970,1,1,0.00000,0.50,0.00000,67758.99998,7.21,67804.00004,67812.99998,9.25,67825.99999
6,326,c37b770b,1,3,0.00000,5.38,67792.00003,0.00000,2.39,67880.99998,67887.00003,2.18,67892.00000
7,376,c37ba1b6,1,4,0.00000,0.49,67794.00002,0.00000,1.30,67883.99996,67893.99998,4.03,67903.00001
8,602,c37b877b,1,5,0.00000,4.60,67800.99997,67802.99996,10.39,0.00000,67907.99998,3.16,67912.00004
9,427,c37b81a9,1,4,67804.00004,0.10,67806.00003,67813.99998,2.44,67916.00002,67927.00003,3.31,67931.00001


In [23]:
#열별로 결측값 개수 확인
count1=0

for i in df_sort_mcout3:
    count2 = 0
    for j in df_sort_mcout3[i]:
        if j == 0:
            count2+=1
    print('-'*50)
    print(i,"열의 결측값 개수:",count2)
    count1+=count2
    
print('-'*50)
print('전체 결측치:',count1)

--------------------------------------------------
id 열의 결측값 개수: 0
--------------------------------------------------
Uid 열의 결측값 개수: 0
--------------------------------------------------
completed 열의 결측값 개수: 0
--------------------------------------------------
CycleNumber 열의 결측값 개수: 0
--------------------------------------------------
MCIn1 열의 결측값 개수: 112
--------------------------------------------------
MC1Prtime 열의 결측값 개수: 2
--------------------------------------------------
MCOut1 열의 결측값 개수: 23
--------------------------------------------------
MCIn2 열의 결측값 개수: 66
--------------------------------------------------
MC2Prtime 열의 결측값 개수: 0
--------------------------------------------------
MCOut2 열의 결측값 개수: 65
--------------------------------------------------
MCIn3 열의 결측값 개수: 5
--------------------------------------------------
MC3Prtime 열의 결측값 개수: 3
--------------------------------------------------
MCOut3 열의 결측값 개수: 0
--------------------------------------------------
전체 결측치: 276


#### MCIn3 결측치 채우기

In [321]:
check_minmax_ran(df_sort_mcout3, 'MCIn3', ['MCIn1','MCOut1','MCIn2','MCOut2'], ['MCOut3'])

--------------------------------------------------
121 행 입니다.
up: 71186.99999
left: 71282.0
right: 71323.99998
down: 71332.0


In [322]:
check_col_order(df_sort_mcout3,'MCIn3')

----------------------------------------------------------------------
열 내 순서 확인 시작
마지막 행( 280 )은 비교불가
이상값: 0
열 내 순서 확인 완료
----------------------------------------------------------------------


In [59]:
check_col_order_del4(df_sort_mcout3,'MCIn3')

----------------------------------------------------------------------
제거 시작
인덱스- 208 209 210 제거
마지막 행( 280 )은 비교불가
제거 완료
----------------------------------------------------------------------


In [62]:
#열별로 결측값 개수 확인
count1=0

for i in df_sort_mcout3:
    count2 = 0
    for j in df_sort_mcout3[i]:
        if j == 0:
            count2+=1
    print('-'*50)
    print(i,"열의 결측값 개수:",count2)
    count1+=count2
    
print('-'*50)
print('전체 결측치:',count1)

--------------------------------------------------
id 열의 결측값 개수: 0
--------------------------------------------------
Uid 열의 결측값 개수: 0
--------------------------------------------------
completed 열의 결측값 개수: 0
--------------------------------------------------
CycleNumber 열의 결측값 개수: 0
--------------------------------------------------
MCIn1 열의 결측값 개수: 112
--------------------------------------------------
MC1Prtime 열의 결측값 개수: 2
--------------------------------------------------
MCOut1 열의 결측값 개수: 23
--------------------------------------------------
MCIn2 열의 결측값 개수: 66
--------------------------------------------------
MC2Prtime 열의 결측값 개수: 0
--------------------------------------------------
MCOut2 열의 결측값 개수: 65
--------------------------------------------------
MCIn3 열의 결측값 개수: 0
--------------------------------------------------
MC3Prtime 열의 결측값 개수: 3
--------------------------------------------------
MCOut3 열의 결측값 개수: 0
--------------------------------------------------
전체 결측치: 271


#### MCOut2 결측치 채우기

In [84]:
#예외
#df_sort_mcout3['MCIn2'][247] = 74180

In [315]:
check_minmax_ran(df_sort_mcout3, 'MCOut2', ['MCIn1','MCOut1','MCIn2'], ['MCIn3','MCOut3'])

--------------------------------------------------
121 행 입니다.
up: 71163.0
left: 71280.0
right: 71254.99999
down: 71293.0
해당 행은 무시( 1 )


In [316]:
check_col_order(df_sort_mcout3,'MCOut2')

----------------------------------------------------------------------
열 내 순서 확인 시작
마지막 행( 280 )은 비교불가
이상값: 0
열 내 순서 확인 완료
----------------------------------------------------------------------


In [283]:
check_col_order_del_updown(df_sort_mcout3,'MCOut2')

----------------------------------------------------------------------
제거 시작
마지막 행( 280 )은 비교불가
제거 완료
----------------------------------------------------------------------


In [284]:
df_sort_mcout3

,id,Uid,completed,CycleNumber,MCIn1,MC1Prtime,MCOut1,MCIn2,MC2Prtime,MCOut2,MCIn3,MC3Prtime,MCOut3
0,401,c37b9987,1,4,67710.00004,2.59,67714.00001,67717.99999,0.22,67721.99996,67731.99998,1.71,67733.99997
1,601,c37b877b,1,5,67721.99996,4.38,67728.00001,67729.00000,2.52,67733.00000,67753.00002,8.22,67762.99996
2,426,c37b81a9,1,4,67728.00000,4.91,67734.99996,67738.00003,1.18,67739.00000,67767.00002,2.30,67772.21996
3,1,c37ba1af,1,1,67740.00000,0.13,67754.00002,67756.00000,4.18,67771.00000,67789.99996,0.08,67792.00003
4,51,c37b9728,1,1,67743.00000,10.40,67778.00000,67786.00000,0.62,67790.00000,67809.00001,1.61,67811.99999
5,76,c37b7970,1,1,67750.00000,0.50,67787.00000,67802.00000,7.21,67804.00004,67812.99998,9.25,67825.99999
6,326,c37b770b,1,3,67762.00000,5.38,67792.00000,67810.00000,2.39,67880.99998,67887.00003,2.18,67892.00000
7,376,c37ba1b6,1,4,67769.00000,0.49,67794.00002,67881.00000,1.30,67883.99996,67893.99998,4.03,67903.00001
8,602,c37b877b,1,5,67771.00000,4.60,67800.99997,67884.00000,10.39,67898.00000,67907.99998,3.16,67912.00004
9,427,c37b81a9,1,4,67804.00004,0.10,67806.00003,67906.00000,2.44,67916.00002,67927.00003,3.31,67931.00001


In [285]:
#열별로 결측값 개수 확인
count1=0

for i in df_sort_mcout3:
    count2 = 0
    for j in df_sort_mcout3[i]:
        if j == 0:
            count2+=1
    print('-'*50)
    print(i,"열의 결측값 개수:",count2)
    count1+=count2
    
print('-'*50)
print('전체 결측치:',count1)

--------------------------------------------------
id 열의 결측값 개수: 0
--------------------------------------------------
Uid 열의 결측값 개수: 0
--------------------------------------------------
completed 열의 결측값 개수: 7
--------------------------------------------------
CycleNumber 열의 결측값 개수: 0
--------------------------------------------------
MCIn1 열의 결측값 개수: 0
--------------------------------------------------
MC1Prtime 열의 결측값 개수: 2
--------------------------------------------------
MCOut1 열의 결측값 개수: 0
--------------------------------------------------
MCIn2 열의 결측값 개수: 0
--------------------------------------------------
MC2Prtime 열의 결측값 개수: 0
--------------------------------------------------
MCOut2 열의 결측값 개수: 0
--------------------------------------------------
MCIn3 열의 결측값 개수: 0
--------------------------------------------------
MC3Prtime 열의 결측값 개수: 3
--------------------------------------------------
MCOut3 열의 결측값 개수: 0
--------------------------------------------------
전체 결측치: 12


#### MCIn2 결측치 채우기

In [133]:
#예외
df_sort_mcout3['MCIn2'][45] = 68600

In [309]:
t = 10
while t != 0:
    check_minmax_ran(df_sort_mcout3, 'MCIn2', ['MCIn1','MCOut1'], ['MCOut2','MCIn3','MCOut3'])
    check_col_order(df_sort_mcout3,'MCIn2')
    check_col_order_del(df_sort_mcout3,'MCIn2')
    t -= 1

--------------------------------------------------
125 행 입니다.
up: 71370.0
left: 71406.0
right: 71417.0
down: 71408.0
----------------------------------------------------------------------
열 내 순서 확인 시작
마지막 행( 280 )은 비교불가
이상값: 0
열 내 순서 확인 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
제거 시작
마지막 행( 280 )은 비교불가
제거 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
열 내 순서 확인 시작
마지막 행( 280 )은 비교불가
이상값: 0
열 내 순서 확인 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
제거 시작
마지막 행( 280 )은 비교불가
제거 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
열 내 순서 확인 시작
마지막 행( 280 )은 비교불가
이상값: 0
열 내 순서 확인 완료
--------------------------

In [310]:
#열별로 결측값 개수 확인
count1=0

for i in df_sort_mcout3:
    count2 = 0
    for j in df_sort_mcout3[i]:
        if j == 0:
            count2+=1
    print('-'*50)
    print(i,"열의 결측값 개수:",count2)
    count1+=count2
    
print('-'*50)
print('전체 결측치:',count1)

--------------------------------------------------
id 열의 결측값 개수: 0
--------------------------------------------------
Uid 열의 결측값 개수: 0
--------------------------------------------------
completed 열의 결측값 개수: 7
--------------------------------------------------
CycleNumber 열의 결측값 개수: 0
--------------------------------------------------
MCIn1 열의 결측값 개수: 0
--------------------------------------------------
MC1Prtime 열의 결측값 개수: 2
--------------------------------------------------
MCOut1 열의 결측값 개수: 0
--------------------------------------------------
MCIn2 열의 결측값 개수: 0
--------------------------------------------------
MC2Prtime 열의 결측값 개수: 0
--------------------------------------------------
MCOut2 열의 결측값 개수: 3
--------------------------------------------------
MCIn3 열의 결측값 개수: 0
--------------------------------------------------
MC3Prtime 열의 결측값 개수: 3
--------------------------------------------------
MCOut3 열의 결측값 개수: 0
--------------------------------------------------
전체 결측치: 15


#### MCOut1 결측치 채우기

In [298]:
t=10
while t != 0:
    check_minmax_ran(df_sort_mcout3, 'MCOut1', ['MCIn1'], ['MCIn2','MCOut2','MCIn3','MCOut3'])
    check_col_order(df_sort_mcout3,'MCOut1')
    check_col_order_del_updown(df_sort_mcout3,'MCOut1')
    t -= 1

----------------------------------------------------------------------
열 내 순서 확인 시작
마지막 행( 280 )은 비교불가
이상값: 0
열 내 순서 확인 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
제거 시작
마지막 행( 280 )은 비교불가
제거 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
열 내 순서 확인 시작
마지막 행( 280 )은 비교불가
이상값: 0
열 내 순서 확인 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
제거 시작
마지막 행( 280 )은 비교불가
제거 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
열 내 순서 확인 시작
마지막 행( 280 )은 비교불가
이상값: 0
열 내 순서 확인 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
제

In [274]:
#열별로 결측값 개수 확인
count1=0

for i in df_sort_mcout3:
    count2 = 0
    for j in df_sort_mcout3[i]:
        if j == 0:
            count2+=1
    print('-'*50)
    print(i,"열의 결측값 개수:",count2)
    count1+=count2
    
print('-'*50)
print('전체 결측치:',count1)

--------------------------------------------------
id 열의 결측값 개수: 0
--------------------------------------------------
Uid 열의 결측값 개수: 0
--------------------------------------------------
completed 열의 결측값 개수: 7
--------------------------------------------------
CycleNumber 열의 결측값 개수: 0
--------------------------------------------------
MCIn1 열의 결측값 개수: 0
--------------------------------------------------
MC1Prtime 열의 결측값 개수: 2
--------------------------------------------------
MCOut1 열의 결측값 개수: 0
--------------------------------------------------
MCIn2 열의 결측값 개수: 2
--------------------------------------------------
MC2Prtime 열의 결측값 개수: 0
--------------------------------------------------
MCOut2 열의 결측값 개수: 5
--------------------------------------------------
MCIn3 열의 결측값 개수: 0
--------------------------------------------------
MC3Prtime 열의 결측값 개수: 3
--------------------------------------------------
MCOut3 열의 결측값 개수: 0
--------------------------------------------------
전체 결측치: 19


#### MCIn1 결측치 채우기

In [204]:
check_minmax_ran(df_sort_mcout3, 'MCIn1', [], ['MCOut1','MCIn2','MCOut2','MCIn3','MCOut3'])

--------------------------------------------------
2 행 입니다.
up: 67721.99996
left: 67721.99996
right: 67734.99996
down: 67719.99997
해당 행은 무시( 1 )
--------------------------------------------------
3 행 입니다.
up: 67728.0
left: 67728.0
right: 67753.0
down: 67719.99997
해당 행은 무시( 2 )
--------------------------------------------------
5 행 입니다.
up: 67728.0
left: 67728.0
right: 67754.0
down: 67772.99998
--------------------------------------------------
6 행 입니다.
up: 67750.0
left: 67750.0
right: 67792.00003
down: 67772.99998
--------------------------------------------------
7 행 입니다.
up: 67762.0
left: 67762.0
right: 67794.00002
down: 67772.99998
--------------------------------------------------
8 행 입니다.
up: 67769.0
left: 67769.0
right: 67800.99997
down: 67772.99998
--------------------------------------------------
11 행 입니다.
up: 67815.99996
left: 67815.99996
right: 67831.00004
down: 67772.99998
해당 행은 무시( 3 )
--------------------------------------------------
15 행 입니다.
up: 67915.00002
left: 67915

In [275]:
t=100
while t != 0:
    check_minmax_ran(df_sort_mcout3, 'MCIn1', [], ['MCOut1','MCIn2','MCOut2','MCIn3','MCOut3'])
    check_col_order(df_sort_mcout3,'MCIn1')
    check_col_order_del_updown(df_sort_mcout3,'MCIn1')
    t -= 1

----------------------------------------------------------------------
열 내 순서 확인 시작
마지막 행( 280 )은 비교불가
이상값: 0
열 내 순서 확인 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
제거 시작
마지막 행( 280 )은 비교불가
제거 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
열 내 순서 확인 시작
마지막 행( 280 )은 비교불가
이상값: 0
열 내 순서 확인 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
제거 시작
마지막 행( 280 )은 비교불가
제거 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
열 내 순서 확인 시작
마지막 행( 280 )은 비교불가
이상값: 0
열 내 순서 확인 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
제

마지막 행( 280 )은 비교불가
제거 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
열 내 순서 확인 시작
마지막 행( 280 )은 비교불가
이상값: 0
열 내 순서 확인 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
제거 시작
마지막 행( 280 )은 비교불가
제거 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
열 내 순서 확인 시작
마지막 행( 280 )은 비교불가
이상값: 0
열 내 순서 확인 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
제거 시작
마지막 행( 280 )은 비교불가
제거 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
열 내 순서 확인 시작
마지막 행( 280 )은 비교불가
이상값: 0
열 내 순서 확인 완료
-----------------------------------------------

마지막 행( 280 )은 비교불가
제거 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
열 내 순서 확인 시작
마지막 행( 280 )은 비교불가
이상값: 0
열 내 순서 확인 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
제거 시작
마지막 행( 280 )은 비교불가
제거 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
열 내 순서 확인 시작
마지막 행( 280 )은 비교불가
이상값: 0
열 내 순서 확인 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
제거 시작
마지막 행( 280 )은 비교불가
제거 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
열 내 순서 확인 시작
마지막 행( 280 )은 비교불가
이상값: 0
열 내 순서 확인 완료
-----------------------------------------------

마지막 행( 280 )은 비교불가
이상값: 0
열 내 순서 확인 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
제거 시작
마지막 행( 280 )은 비교불가
제거 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
열 내 순서 확인 시작
마지막 행( 280 )은 비교불가
이상값: 0
열 내 순서 확인 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
제거 시작
마지막 행( 280 )은 비교불가
제거 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
열 내 순서 확인 시작
마지막 행( 280 )은 비교불가
이상값: 0
열 내 순서 확인 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
제거 시작
마지막 행( 280 )은 비교불가
제거 완료
------------------------------------------------------

In [324]:
#열별로 결측값 개수 확인
count1=0

for i in df_sort_mcout3:
    count2 = 0
    for j in df_sort_mcout3[i]:
        if j == 0:
            count2+=1
    print('-'*50)
    print(i,"열의 결측값 개수:",count2)
    count1+=count2
    
print('-'*50)
print('전체 결측치:',count1)

--------------------------------------------------
id 열의 결측값 개수: 0
--------------------------------------------------
Uid 열의 결측값 개수: 0
--------------------------------------------------
completed 열의 결측값 개수: 7
--------------------------------------------------
CycleNumber 열의 결측값 개수: 1
--------------------------------------------------
MCIn1 열의 결측값 개수: 0
--------------------------------------------------
MC1Prtime 열의 결측값 개수: 2
--------------------------------------------------
MCOut1 열의 결측값 개수: 0
--------------------------------------------------
MCIn2 열의 결측값 개수: 0
--------------------------------------------------
MC2Prtime 열의 결측값 개수: 0
--------------------------------------------------
MCOut2 열의 결측값 개수: 0
--------------------------------------------------
MCIn3 열의 결측값 개수: 0
--------------------------------------------------
MC3Prtime 열의 결측값 개수: 3
--------------------------------------------------
MCOut3 열의 결측값 개수: 0
--------------------------------------------------
전체 결측치: 13


In [249]:
df_dropped_resetIdx = df_sort_mcout3
df_dropped_resetIdx

,id,Uid,completed,CycleNumber,MCIn1,MC1Prtime,MCOut1,MCIn2,MC2Prtime,MCOut2,MCIn3,MC3Prtime,MCOut3
0,401,c37b9987,1,4,67710.00004,2.59,67714.00001,67717.99999,0.22,67721.99996,67731.99998,1.71,67733.99997
1,601,c37b877b,1,5,67721.99996,4.38,67728.00001,67729.00000,2.52,67733.00000,67753.00002,8.22,67762.99996
2,426,c37b81a9,1,4,67728.00000,4.91,67734.99996,67738.00003,1.18,67739.00000,67767.00002,2.30,67772.21996
3,1,c37ba1af,1,1,67740.00000,0.13,67754.00002,67756.00000,4.18,67771.00000,67789.99996,0.08,67792.00003
4,51,c37b9728,1,1,67743.00000,10.40,67778.00000,67787.00000,0.62,67777.00000,67809.00001,1.61,67811.99999
5,76,c37b7970,1,1,67750.00000,0.50,67787.00000,67802.00000,7.21,67804.00004,67812.99998,9.25,67825.99999
6,326,c37b770b,1,3,67762.00000,5.38,67792.00000,67810.00000,2.39,67880.99998,67887.00003,2.18,67892.00000
7,376,c37ba1b6,1,4,67769.00000,0.49,67794.00002,67881.00000,1.30,67883.99996,67893.99998,4.03,67903.00001
8,602,c37b877b,1,5,67771.00000,4.60,67800.99997,67884.00000,10.39,67898.00000,67907.99998,3.16,67912.00004
9,427,c37b81a9,1,4,67804.00004,0.10,67806.00003,67906.00000,2.44,67916.00002,67927.00003,3.31,67931.00001


### 이상치 제거한 version 재확인

#### 행을 돌면서 확인

In [325]:
# 행을 돌면서 m1in<m1out<m2in<m2out<m3in<m3out 구조를 따르지 않는 데이터 값 확인
count = 0
m1inout = 0
m1m2 = 0
m2inout = 0
m2m3 = 0
m3inout = 0

for i, j in enumerate(df_dropped_resetIdx['MCIn1']):
    # 왼쪽 0 오른쪽값 살리기
    # m1in, m1out 비교
    if df_dropped_resetIdx.iloc[i, 6] < df_dropped_resetIdx.iloc[i, 4]:
        if (df_dropped_resetIdx.iloc[i, 1] != 4) & (df_dropped_resetIdx.iloc[i, 6] != 0):
            df_dropped_resetIdx.iloc[i, 6] = 0
            print('확인바람: ',i , '행 m1in, m1out')
            count+=1
            m1inout+=1
    # m1out, m2in 비교
    if df_dropped_resetIdx.iloc[i, 7] < df_dropped_resetIdx.iloc[i, 6]:
        if (df_dropped_resetIdx.iloc[i, 7] != 0) & (df_dropped_resetIdx.iloc[i, 6] != 0):
            df_dropped_resetIdx.iloc[i, 7] = 0
            print('확인바람: ',i , '행 m1out, m2in')
            count+=1
            m1m2 += 1
    # m2in, m2out 비교
    if df_dropped_resetIdx.iloc[i, 9] < df_dropped_resetIdx.iloc[i, 7]:
        if (df_dropped_resetIdx.iloc[i, 9] != 0) & (df_dropped_resetIdx.iloc[i, 7] != 0):
            df_dropped_resetIdx.iloc[i, 9] = 0
            print('확인바람: ',i , '행 m2in, m2out')
            count+=1
            m2inout += 1
    # m2out, m3in 비교
    if df_dropped_resetIdx.iloc[i, 10] < df_dropped_resetIdx.iloc[i, 9]:
        if (df_dropped_resetIdx.iloc[i, 10] != 0) & (df_dropped_resetIdx.iloc[i, 9] != 0):
            df_dropped_resetIdx.iloc[i, 10] = 0
            print('확인바람: ',i , '행 m2out, m3in')
            count+=1
            m2m3 +=1
    # m3in, m3out 비교
    if df_dropped_resetIdx.iloc[i, 12] < df_dropped_resetIdx.iloc[i, 10]:
        if (df_dropped_resetIdx.iloc[i, 12] != 0) & (df_dropped_resetIdx.iloc[i, 10] != 0):
            #df_dropped_resetIdx.iloc[i, 4] = 0
            print('확인바람: ',i , '행 m3in, m3out')
            count+=1
            m3inout +=1

print('이상값:',count,', m1inout:',m1inout,", m1m2:",m1m2,", m2inout:",m2inout,", m2m3",m2m3,", m3inout:",m3inout)
df_dropped_resetIdx.head(10)

이상값: 0 , m1inout: 0 , m1m2: 0 , m2inout: 0 , m2m3 0 , m3inout: 0


,id,Uid,completed,CycleNumber,MCIn1,MC1Prtime,MCOut1,MCIn2,MC2Prtime,MCOut2,MCIn3,MC3Prtime,MCOut3
0,401,c37b9987,1,4,67710.00004,2.59,67714.00001,67717.99999,0.22,67721.99996,67731.99998,1.71,67733.99997
1,601,c37b877b,1,5,67721.99996,4.38,67728.00001,67729.00000,2.52,67733.00000,67753.00002,8.22,67762.99996
2,426,c37b81a9,1,4,67728.00000,4.91,67734.99996,67738.00003,1.18,67739.00000,67767.00002,2.30,67772.21996
3,1,c37ba1af,1,1,67740.00000,0.13,67754.00002,67756.00000,4.18,67771.00000,67789.99996,0.08,67792.00003
4,51,c37b9728,1,1,67743.00000,10.40,67778.00000,67786.00000,0.62,67790.00000,67809.00001,1.61,67811.99999
5,76,c37b7970,1,1,67750.00000,0.50,67787.00000,67802.00000,7.21,67804.00004,67812.99998,9.25,67825.99999
6,326,c37b770b,1,3,67762.00000,5.38,67792.00000,67810.00000,2.39,67880.99998,67887.00003,2.18,67892.00000
7,376,c37ba1b6,1,4,67769.00000,0.49,67794.00002,67881.00000,1.30,67883.99996,67893.99998,4.03,67903.00001
8,602,c37b877b,1,5,67771.00000,4.60,67800.99997,67884.00000,10.39,67898.00000,67907.99998,3.16,67912.00004
9,427,c37b81a9,1,4,67804.00004,0.10,67806.00003,67906.00000,2.44,67916.00002,67927.00003,3.31,67931.00001


#### 열을 돌면서 확인

In [326]:
happy = ['MCIn1', 'MCOut1', 'MCIn2', 'MCOut2', 'MCIn3', 'MCOut3']
for i in happy:
    check_col_order(df_dropped_resetIdx,i)

----------------------------------------------------------------------
열 내 순서 확인 시작
마지막 행( 280 )은 비교불가
이상값: 0
열 내 순서 확인 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
열 내 순서 확인 시작
마지막 행( 280 )은 비교불가
이상값: 0
열 내 순서 확인 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
열 내 순서 확인 시작
마지막 행( 280 )은 비교불가
이상값: 0
열 내 순서 확인 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
열 내 순서 확인 시작
마지막 행( 280 )은 비교불가
이상값: 0
열 내 순서 확인 완료
----------------------------------------------------------------------
----------------------------------------------------------------------
열 내 순서 확인 시작
마지막 행( 280 )은 비교불가
이상값: 0
열 내 순서 확인 완료
----------------------------------------------------------------------
------------------------------